In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from helper_functions import *
import os

In [2]:
LR_data = pd.read_csv("./LR_Model_Predictions_(2016-2018).csv", low_memory=False, header=0)
LGBM_data = pd.read_csv("./LGBM_Predictions.csv", low_memory=False, header=0)
LSTM_data = pd.read_csv("./LSTM_predicted.csv", low_memory=False, header=0)
RF_data = pd.read_csv("./RF_predictions.csv", low_memory=False, header=0)
SARIMA_data = pd.read_csv("./SARIMA_Predictions.csv", low_memory=False, header=0)

In [3]:
# Standardize format
SARIMA_data.rename(columns=lambda x: x + "_SARIMA", inplace=True)
SARIMA_data.rename(columns={"Unnamed: 0_SARIMA":'DATE'}, inplace=True)
LGBM_data.rename(columns={"dates":'DATE'}, inplace=True)
LR_data["DATE"] = list(map(str, LR_data["DATE"]))
LR_data["DATE"] = LR_data["DATE"].apply(lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:8])

result = LGBM_data.set_index("DATE").join(LSTM_data.set_index("DATE"), on='DATE', lsuffix="_LGBM", rsuffix="_LSTM").reset_index()
result = result.set_index("DATE").join(RF_data.set_index("DATE"), on='DATE').reset_index()
result = result.set_index("DATE").join(LR_data.set_index("DATE"), on='DATE', lsuffix="_RF", rsuffix="_LR").reset_index()
result = result.set_index("DATE").join(SARIMA_data.set_index("DATE"), on='DATE').reset_index()

result = result.fillna(method='bfill')
result.head()

result = result[result.columns.drop(list(result.filter(regex='LSTM')))]
result.head()

,DATE,F_AD_LGBM,F_AE_LGBM,F_AH_LGBM,F_AX_LGBM,F_BC_LGBM,F_BG_LGBM,F_BO_LGBM,F_BP_LGBM,F_C_LGBM,...,F_US_SARIMA,F_UZ_SARIMA,F_VF_SARIMA,F_VT_SARIMA,F_VW_SARIMA,F_VX_SARIMA,F_W_SARIMA,F_XX_SARIMA,F_YM_SARIMA,F_ZQ_SARIMA
0,2016-01-01,71296.911747,85901.674488,7829.242385,255484.549448,38365.514703,34013.656915,18215.556032,92581.499364,17829.692822,...,153750.0,111510.548466,128006.370234,147888.184302,112052.738638,18504.964587,23515.010887,30928.682762,86795.103351,414913.543865
1,2016-01-04,71296.911747,85901.674488,7829.242385,255484.549448,38365.514703,34013.656915,18215.556032,92581.499364,17829.692822,...,153750.0,111509.301163,128012.740785,147902.675392,112054.282462,18489.671922,23513.234782,30921.906748,86839.477543,414918.441300
2,2016-01-05,71296.911747,85901.674488,7829.242385,255484.549448,38365.514703,34013.656915,18215.556032,91997.118490,17829.692822,...,153750.0,111510.124515,128019.111653,147920.889259,112057.243954,18477.996683,23497.890472,30917.849244,86839.477543,414923.338792
3,2016-01-06,71050.233873,86663.424730,7829.242385,255484.549448,37032.812581,33906.859206,18056.948621,91401.078401,17792.318272,...,153750.0,111509.399341,128025.482838,147933.574103,112059.640625,18469.081644,23489.864373,30915.419424,86839.477543,414928.236342
4,2016-01-07,70580.112371,86786.813500,7829.242385,254367.179741,35499.680274,32161.040913,18000.390457,91797.971859,17730.697397,...,153750.0,111510.077885,128031.854340,147946.260034,112062.037346,18462.273359,23499.314896,30913.964274,86839.477543,414933.133950


In [4]:
all_x_train = result.loc[0:520,:]
all_x_test = result.loc[521:,:]

all_y_train = pd.read_pickle("data_base_test.pkl")
all_y_train = all_y_train.fillna(method='bfill')

all_y_test = pd.read_pickle("data_stack_test.pkl")
all_y_test = all_y_test.fillna(method='bfill')

/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)
/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'co

In [5]:
futures = ['F_AD','F_AE','F_AH','F_AX','F_BC','F_BG','F_BO','F_BP',
               'F_C','F_CA','F_CC','F_CD','F_CF','F_CL','F_CT','F_DL',             
               'F_DM','F_DT','F_DX','F_DZ','F_EB','F_EC','F_ED','F_ES',
               'F_F','F_FB','F_FC','F_FL','F_FM','F_FP','F_FV','F_FY',             
               'F_GC','F_GD','F_GS','F_GX','F_HG','F_HO','F_HP','F_JY',
              'F_KC','F_LB','F_LC','F_LN','F_LQ','F_LR','F_LU','F_LX',
           'F_MD','F_MP','F_ND','F_NG','F_NQ','F_NR','F_NY','F_O',
             'F_OJ','F_PA','F_PL','F_PQ','F_RB','F_RF','F_RP','F_RR',
               'F_RU','F_RY','F_S','F_SB','F_SF','F_SH','F_SI','F_SM',
             'F_SS','F_SX','F_TR','F_TU','F_TY','F_UB','F_US','F_UZ',
            'F_VX','F_W','F_XX','F_YM','F_ZQ']

In [6]:
final_preds = pd.DataFrame(index=np.arange(len(all_x_test)), columns=np.arange(len(futures)))
final_preds.columns = futures
final_preds["DATE"] = all_x_test["DATE"].values
final_preds = final_preds.set_index("DATE")
final_preds.head()

,F_AD,F_AE,F_AH,F_AX,F_BC,F_BG,F_BO,F_BP,F_C,F_CA,...,F_TU,F_TY,F_UB,F_US,F_UZ,F_VX,F_W,F_XX,F_YM,F_ZQ
DATE,,,,,,,,,,,,,,,,,,,,,
2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
result = pd.DataFrame(columns=['LGBM', 'RF', 'LR', 'SARIMA', 'Intercept', 'MAPE'])
for fut in futures:
    y_train = all_y_train[fut]['CLOSE']
    y_train = pd.DataFrame(y_train)
    y_train = y_train.fillna(method='ffill')
    y_test = all_y_test[fut]['CLOSE']
    y_test = pd.DataFrame(y_test)
    y_test = y_test.fillna(method='ffill')
    tempfut = fut + '_'
    feature_list = []
    for j in all_x_train.columns:
        if tempfut in j:
            feature_list.append(j)
    x_train = all_x_train.loc[:,feature_list]
    x_train = x_train.fillna(method='ffill')
    x_test = all_x_test.loc[:,feature_list]
    x_test = x_test.fillna(method='ffill')
    lr = LinearRegression()
    model = lr.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    #Store predictions 
    for i in range(len(final_preds.index.values)):
        final_preds.at[final_preds.index.values[i], fut] = y_pred[i][0]
    
    coefficient = lr.coef_
    intercept = lr.intercept_
    rmse = rmse_ratio(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    print(coefficient)
    print(intercept)
    print(mape)
    result2 = pd.DataFrame(coefficient, columns=['LGBM', 'RF', 'LR', 'SARIMA'])
    result2['Intercept'] = intercept
    result2['MAPE'] = mape.values
    result = result.append(result2, ignore_index=True)
    #print('MAPE for ', fut, ': ', mean_absolute_percentage_error(y_test, y_pred))  
    #print('RMSE for ', fut, ': ', rmse_ratio(y_test, y_pred))

[[-0.03002891  0.24021945  0.81971372 -0.05231094]]
[1708.33116111]
CLOSE    0.424463
dtype: float64
[[-0.00459667  0.22883002  0.78593019 -0.01416253]]
[463.03667759]
CLOSE    0.690916
dtype: float64
[[ 0.0191138  -0.02197009  1.00105826 -0.03018747]]
[267.63094748]
CLOSE    0.573408
dtype: float64
[[-0.09034119  0.11864512  1.02372625 -0.03573409]]
[-3955.09293809]
CLOSE    0.785713
dtype: float64
[[-0.11191385  0.16869449  0.93860394 -0.0081509 ]]
[715.28788682]
CLOSE    1.370818
dtype: float64
[[-0.14847408  0.09824373  1.0723839  -0.03039359]]
[453.98790347]
CLOSE    1.156365
dtype: float64
[[ 0.07176049  0.10971218  0.83959846 -0.04864122]]
[554.62147986]
CLOSE    0.825547
dtype: float64
[[ 0.01560999 -0.09691875  1.04089899 -0.03268412]]
[6388.60555165]
CLOSE    0.40489
dtype: float64
[[0.01151693 0.11014017 0.81977126 0.03222724]]
[473.38645432]
CLOSE    0.81884
dtype: float64
[[-0.08112985  0.27630137  0.85183232 -0.05899121]]
[561.32491491]
CLOSE    0.64957
dtype: float64
[[-

In [8]:
#final_preds.to_csv("Final_Predictions.csv", index=False)

In [9]:
all_x_test.head()

,DATE,F_AD_LGBM,F_AE_LGBM,F_AH_LGBM,F_AX_LGBM,F_BC_LGBM,F_BG_LGBM,F_BO_LGBM,F_BP_LGBM,F_C_LGBM,...,F_US_SARIMA,F_UZ_SARIMA,F_VF_SARIMA,F_VT_SARIMA,F_VW_SARIMA,F_VX_SARIMA,F_W_SARIMA,F_XX_SARIMA,F_YM_SARIMA,F_ZQ_SARIMA
521,2018-01-01,78333.293563,108571.222862,8869.279471,300757.296494,66456.907141,60124.632877,20154.452512,87035.878729,17730.697397,...,152829.056052,111965.471770,128042.742810,149521.873891,112275.664092,11709.052523,21329.004090,31327.815586,123495.950417,410570.447990
522,2018-01-02,78333.293563,108571.222862,8869.279471,300757.296494,66456.907141,60124.632877,20154.452512,87035.878729,17730.697397,...,152829.056052,111965.232796,128049.114691,149296.046776,112280.520332,11871.726350,21326.882213,31282.937254,123841.363500,410574.661099
523,2018-01-03,78333.293563,108571.222862,8869.279471,300757.296494,66456.907141,60124.632877,20154.452512,87035.878729,17730.697397,...,152829.056052,111967.366831,128055.486889,149179.860259,112282.974566,11999.133222,21360.108730,31255.354084,123841.363500,410578.874251
524,2018-01-04,78344.670161,109788.332420,8869.279471,300757.296494,67377.920132,61287.502726,20211.633309,87035.878729,17792.318272,...,152829.056052,111964.861935,128061.859404,149277.234753,112285.428853,12098.643035,21383.326464,31238.393334,123841.363500,410583.087447
525,2018-01-05,78657.706831,110512.910287,8869.279471,300757.296494,67433.674507,61366.069844,20307.890995,87035.878729,17437.864602,...,152160.276611,111947.884533,128066.380511,149470.539853,112282.004619,10948.925622,21700.000000,31636.431942,124503.475643,410499.481935


In [10]:
result

,LGBM,RF,LR,SARIMA,Intercept,MAPE
0,-0.030029,0.240219,0.819714,-0.052311,1708.331161,0.424463
1,-0.004597,0.228830,0.785930,-0.014163,463.036678,0.690916
2,0.019114,-0.021970,1.001058,-0.030187,267.630947,0.573408
3,-0.090341,0.118645,1.023726,-0.035734,-3955.092938,0.785713
4,-0.111914,0.168694,0.938604,-0.008151,715.287887,1.370818
5,-0.148474,0.098244,1.072384,-0.030394,453.987903,1.156365
6,0.071760,0.109712,0.839598,-0.048641,554.621480,0.825547
7,0.015610,-0.096919,1.040899,-0.032684,6388.605552,0.404890
8,0.011517,0.110140,0.819771,0.032227,473.386454,0.818840
9,-0.081130,0.276301,0.851832,-0.058991,561.324915,0.649570


In [11]:
result.to_csv("Stacked_Model_Coefficients(WITHOUT LSTM).csv", index=False)